# Table of Contents
* [Target Exploration](#1)
* [Features](#2)
* [Fit Model](#3)
* [Evaluate Model](#4)

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd
import time

# plots
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

# machine learning tools
import h2o
from h2o.estimators import H2OGradientBoostingEstimator

In [ ]:
# load data + first glance
df_train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
df_sub = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

# first glance (training data)
df_train.head()

In [ ]:
# dimensions
print('Train Set:', df_train.shape)
print('Test Set :', df_test.shape)

In [ ]:
# structure
df_train.info()

<a id='1'></a>
# Target Exploration

In [ ]:
# basic stats/plot
print(df_train.target.value_counts())
df_train.target.value_counts().sort_index().plot(kind='bar')
plt.grid()
plt.show()

In [ ]:
# plot sorted
df_train.target.value_counts().plot(kind='bar')
plt.grid()
plt.show()

<a id='2'></a>
# Features

In [ ]:
# extract features from column names
features = df_train.columns.tolist()
features.remove('id')
features.remove('target')

In [ ]:
# basic summary stats
pd.set_option('display.max_columns', None) # show all columns
df_train[features].describe()

In [ ]:
# plot all features in one plot
df_train.boxplot(column=features, figsize=(16,6))
plt.xticks(rotation=90)
plt.title('Boxplot of features')
plt.show()

In [ ]:
# correlation of features
corr_pearson = df_train[features].corr(method='pearson')
plt.figure(figsize=(12,12))
sns.heatmap(corr_pearson, annot=False, cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Pearson Correlation')
plt.show()

#### We observe only very weak correlations between the features...

In [ ]:
print('Maximum correlation:', np.round(corr_pearson[corr_pearson!=1].max().max(),5))
print('Minimum correlation:', np.round(corr_pearson[corr_pearson!=1].min().min(),5))

#### Let's extract the highest correlations:

In [ ]:
# create data frame to store all results
n_features = len(features)
cor_stats = pd.DataFrame(data=np.zeros((n_features**2,4)), columns=['x','y','corr','sel'])
cor_stats.x = cor_stats.x.astype(str)
cor_stats.y = cor_stats.y.astype(str)

# calc and store all correlations in data frame
count = 1 # count correlations exceeding threshold
row = 0
for i in range(n_features):
    var_i = features[i]
    for j in range(n_features):
        var_j = features[j]            
        cor_x = df_train[var_i].corr(df_train[var_j])
        # store results
        cor_stats.loc[row,'x'] = var_i
        cor_stats.loc[row,'y'] = var_j
        cor_stats.loc[row,'corr'] = cor_x
        if (i>j):
            cor_stats.loc[row,'sel'] = 1 # we use this to later remove redundancies
                
        row = row + 1

# remove redundancies
cor_stats = cor_stats[cor_stats.sel==1] # only select "i > j" cases
cor_stats = cor_stats.drop(['sel'], axis=1)
# sort by correlation (descending)
cor_stats = cor_stats.sort_values(by=['corr'], ascending=False)
cor_stats = cor_stats.reset_index(drop=True)

In [ ]:
# show top 10 correlations
cor_stats.head(10)

<a id='3'></a>
# Fit Model

In [ ]:
# select predictors
predictors = features
print('Number of predictors: ', len(predictors))

In [ ]:
# start H2O
h2o.init(max_mem_size='12G', nthreads=4) # Use maximum of 12 GB RAM and 4 cores

In [ ]:
# upload data frames in H2O environment
t1 = time.time()
train_hex = h2o.H2OFrame(df_train)
test_hex = h2o.H2OFrame(df_test)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

# force categorical target
train_hex['target'] = train_hex['target'].asfactor()

In [ ]:
# fit Gradient Boosting model
n_cv = 5

fit_GBM = H2OGradientBoostingEstimator(ntrees=750,
                                       max_depth=6,
                                       min_rows=25,
                                       learn_rate=0.01, # default: 0.1
                                       sample_rate=1,
                                       col_sample_rate=0.5,
                                       nfolds=n_cv,
                                       score_each_iteration=True,
                                       stopping_metric='logloss',
                                       stopping_rounds=5,
                                       stopping_tolerance=0.0001,
                                       seed=999)
# train model
t1 = time.time()
fit_GBM.train(x=predictors,
              y='target',
              training_frame=train_hex)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

In [ ]:
# show cross validation metrics
fit_GBM.cross_validation_metrics_summary()

In [ ]:
# show scoring history - training vs cross validations
for i in range(n_cv):
    cv_model_temp = fit_GBM.cross_validation_models()[i]
    df_cv_score_history = cv_model_temp.score_history()
    my_title = 'CV ' + str(1+i) + ' - Scoring History'
    plt.scatter(df_cv_score_history.number_of_trees,
                y=df_cv_score_history.training_logloss, 
                c='blue', label='training')
    plt.scatter(df_cv_score_history.number_of_trees,
                y=df_cv_score_history.validation_logloss, 
                c='darkorange', label='validation')
    plt.title(my_title)
    plt.xlabel('Number of Trees')
    plt.ylabel('logloss')
    plt.ylim(1.5,2.5)
    plt.legend()
    plt.grid()
    plt.show()

<a id='4'></a>
# Evaluate Model

In [ ]:
# variable importance
fit_GBM.varimp_plot(25)

### Evaluate Predictions on Training Data

In [ ]:
# predict on train set
pred_train_GBM = fit_GBM.predict(train_hex).as_data_frame()
# add ground truth
pred_train_GBM['target'] = train_hex['target'].as_data_frame()
pred_train_GBM.head()

In [ ]:
# predicted frequencies
classes = ['Class_1', 'Class_2', 'Class_3',
           'Class_4', 'Class_5', 'Class_6',
           'Class_7', 'Class_8', 'Class_9']

pred_train_GBM[classes].sum()

In [ ]:
# actual frequencies
df_train.target.value_counts().sort_index()

In [ ]:
# confusion matrix - training data
conf_train = pd.crosstab(pred_train_GBM.target, pred_train_GBM.predict)
sns.heatmap(conf_train, cmap='Blues',
            annot=True, fmt='d',
            vmin=0, vmax=40000,
            linecolor='black',
            linewidths=0.1)
plt.title('Confusion Matrix - Training')
plt.show()

### Predict on test set

In [ ]:
# predict on test set
pred_test_GBM = fit_GBM.predict(test_hex).as_data_frame()
pred_test_GBM

In [ ]:
# submission
df_sub_GBM = df_sub.copy()
df_sub_GBM.Class_1 = pred_test_GBM.Class_1
df_sub_GBM.Class_2 = pred_test_GBM.Class_2
df_sub_GBM.Class_3 = pred_test_GBM.Class_3
df_sub_GBM.Class_4 = pred_test_GBM.Class_4
df_sub_GBM.Class_5 = pred_test_GBM.Class_5
df_sub_GBM.Class_6 = pred_test_GBM.Class_6
df_sub_GBM.Class_7 = pred_test_GBM.Class_7
df_sub_GBM.Class_8 = pred_test_GBM.Class_8
df_sub_GBM.Class_9 = pred_test_GBM.Class_9
df_sub_GBM

In [ ]:
# export submission
df_sub_GBM.to_csv('submission_GBM.csv', index=False)

In [ ]:
# multi-dimensional visualization of submission
sns.pairplot(df_sub_GBM[classes],
             diag_kws = {'alpha': 1.0},
             plot_kws = {'alpha': 0.1})
plt.show()